In [19]:
#import soundfile as sf
#data, samplerate = sf.read(filename)
#sf.write("C:/Users/sgrebenkin/Downloads/record-normalized.wav", data, 16000)

import subprocess
src_filename = 'C:/Users/sgrebenkin/Downloads/voice_08-11-2024_09-15-56.ogg'
dest_filename = 'C:/Users/sgrebenkin/Downloads/record.wav'

process = subprocess.run(['ffmpeg', '-i', src_filename, dest_filename])
if process.returncode != 0:
    raise Exception("Something went wrong")

!sox "C:/Users/sgrebenkin/Downloads/record.wav" -r 16k "C:/Users/sgrebenkin/Downloads/record-normalized.wav" norm -0.5 compand 0.3,1 -90,-90,-70,-70,-60,-20,0,0 -5 0 0.2

sox WARN compand: compand clipped 9182 samples; decrease volume?
sox WARN rate: rate clipped 1624 samples; decrease volume?
sox WARN dither: dither clipped 1420 samples; decrease volume?


In [20]:
import torch
import numpy as np
import soundfile as sf
from transformers import WhisperForConditionalGeneration, WhisperProcessor, pipeline
from scipy.io.wavfile import read
from pydub import AudioSegment


torch_dtype = torch.bfloat16 # set your preferred type here 

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
    setattr(torch.distributed, "is_initialized", lambda : False) # monkey patching
device = torch.device(device)

whisper = WhisperForConditionalGeneration.from_pretrained(
    "antony66/whisper-large-v3-russian", torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True,
    attn_implementation="flash_attention_2" #if your GPU supports it
)

processor = WhisperProcessor.from_pretrained("antony66/whisper-large-v3-russian")

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

{'text': 'Лично что надо остаться в рамках Сберы, просто пусть он определится, если он хочет про меня гадости какие-то говорить, тогда как я оттуда уйду? В его интересах, даже если он так не считает, а считает обратно, говорит, что я топовый специалист, чтобы меня скорее куда-то забрали и забыть про меня, если он так считает, что я херовый, доп там, допустим, да. Вот, ну, либо если он там про меня херово говорит, я не ухожу, ну, значит, терпеть меня, вот, работать со мной дальше, пока я сам не захочу уволиться, ну, как-то так.'}


In [24]:
import subprocess
src_filename = 'C:/Users/sgrebenkin/Downloads/voice_08-11-2024_09-15-56.ogg'
dest_filename = 'C:/Users/sgrebenkin/Downloads/record.wav'

process = subprocess.run(['ffmpeg', '-i', src_filename, dest_filename])
if process.returncode != 0:
    raise Exception("Something went wrong")

!sox "C:/Users/sgrebenkin/Downloads/record.wav" -r 16k "C:/Users/sgrebenkin/Downloads/record-normalized.wav" norm -0.5 compand 0.3,1 -90,-90,-70,-70,-60,-20,0,0 -5 0 0.2

# Load the audio file (Ogg Vorbis or other formats)
with open('C:/Users/sgrebenkin/Downloads/record-normalized.wav', 'rb') as f:
    audio_data, sample_rate = sf.read(f)

# If stereo, take the first channel
if audio_data.ndim > 1:
    audio_data = audio_data[:, 0]

# Normalize the audio data
audio_data = audio_data.astype(np.float32)
audio_data /= np.max(np.abs(audio_data))

# Normalize the audio data
audio_data /= np.max(np.abs(audio_data))

# get the transcription
asr = asr_pipeline(audio_data, generate_kwargs={"language": "russian", "max_new_tokens": 256}, return_timestamps=False)
#asr = asr_pipeline(audio_data, generate_kwargs={"max_new_tokens": 256}, return_timestamps=False)

print(asr)

sox WARN compand: compand clipped 9182 samples; decrease volume?
sox WARN rate: rate clipped 1624 samples; decrease volume?
sox WARN dither: dither clipped 1416 samples; decrease volume?


{'text': 'Лично что надо остаться в рамках Сберы, просто пусть он определится, если он хочет про меня гадости какие-то говорить, тогда как я оттуда уйду? В его интересах, даже если он так не считает, а считает обратно, говорит, что я топовый специалист, чтобы меня скорее куда-то забрали и забыли про меня, если он так считает, что я херовый, доп там, допустим, да. Вот, ну, либо если он там про меня херово говорит, я не ухожу, ну, значит, терпеть меня, вот, работать со мной дальше, пока я сам не захочу уволиться, ну, как-то так.'}
